In [1]:
import sys
import os

sys.path.append(sys.path[0]+"/tracker")
sys.path.append(sys.path[0]+"/tracker/model")

from track_anything import TrackingAnything
import random 
import string
from dataset.longdataset import LongVideoDataset
from tools.runner import run_model_on_longdata_set


In [2]:
if os.name == 'posix':
    longdataset_root = '../data.nosync/LongDataset/'

else:
    longdataset_root = 'D:/HADA/data/LongDataset/'


In [3]:
SAM_checkpoint = "./checkpoints/sam_vit_h_4b8939.pth"
if 'HQ' in sys.prefix.split('.')[-1]:SAM_checkpoint = "./checkpoints/sam_hq_vit_h.pth"
xmem_checkpoint = "./checkpoints/XMem-s012.pth"

In [4]:
runtimeargs = {
    'DatasetArgs' :{
        'Dataset' : 'LongDataset',
        'Set' : 'val',
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'both_neg',
            'addArgs1':'CP'
    }
}

VideoLoader = LongVideoDataset(root_folder=longdataset_root, filelist= runtimeargs['DatasetArgs']['Set']+'.txt')
runname = '{}{}{}{}_{}_{}'.format(
    'Refined' if runtimeargs['TrackingAnythingArgs']['use_refinement']  else 'XMEM',
    '_HQ' if 'HQ' in sys.prefix.split('.')[-1] else '_SAM',
    '_'+ runtimeargs['TrackingAnythingArgs']['refinement_mode'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] else '',
    '_' + runtimeargs['TrackingAnythingArgs']['addArgs1'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] and runtimeargs['TrackingAnythingArgs']['addArgs1'] != '' else '',
    'Davis_{}_{}_{}'.format(runtimeargs['DatasetArgs']['Year'],runtimeargs['DatasetArgs']['Set'],runtimeargs['DatasetArgs']['Resolution']) if runtimeargs['DatasetArgs']['Dataset'] == 'Davis' else runtimeargs['DatasetArgs']['Dataset'],
    ''.join(random.choice(string.ascii_letters) for _ in range(5))
)
print(f'Running Test: {runname}')
xmem_checkpoint = 'checkpoints\XMem.pth'
model = TrackingAnything(SAM_checkpoint, xmem_checkpoint, None,runtimeargs['TrackingAnythingArgs'], save_inner_masks_folder = runname)
masks, logits, painted_images = run_model_on_longdata_set(name = runname,model = model,videoLoader = VideoLoader,compute_metrics = True, save_masks=False, compute_video=False,verbose = False)

Seq names: ['rat', 'dressage', 'blueboy']
Running Test: Refined_SAM_both_neg_CP_LongDataset_jsjUq
Initializing BaseSegmenter to cuda:0
Hyperparameters read from the model weights: C^k=64, C^v=512, C^h=64
Single object mode: False
algorithmCP
Sam Refinement ACTIVATED. Mode: both_neg


Tracking image:   2%|▏         | 33/1416 [00:09<06:36,  3.49it/s]

: 